# [Chat with India's Budget 2024 using your local CPU chatbot](https://www.aimplabs.org/blogs/chatbot-budget2024/)

Abhishek Acharya, AIMP LABS  
Published on: 01-Sep-2024  
Last Updated on: 01-Sep-2024

## Prerequisites

In [1]:
!wget +o TinyLlama-1.1B-Chat-v1.0.F16.llamafile https://huggingface.co/Mozilla/TinyLlama-1.1B-Chat-v1.0-llamafile/resolve/main/TinyLlama-1.1B-Chat-v1.0.F16.llamafile

--2024-09-02 20:41:12--  http://+o/
Resolving +o (+o)... failed: Name or service not known.
wget: unable to resolve host address ‘+o’
--2024-09-02 20:41:12--  http://tiny.llamafile/
Resolving tiny.llamafile (tiny.llamafile)... failed: Name or service not known.
wget: unable to resolve host address ‘tiny.llamafile’
--2024-09-02 20:41:12--  https://huggingface.co/Mozilla/TinyLlama-1.1B-Chat-v1.0-llamafile/resolve/main/TinyLlama-1.1B-Chat-v1.0.F16.llamafile
Resolving huggingface.co (huggingface.co)... 18.155.107.35, 18.155.107.105, 18.155.107.90, ...
Connecting to huggingface.co (huggingface.co)|18.155.107.35|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/21/08/210853db4b63019e38fe484022a8fb6fbe3fc6936ec5714947ae77e7296d5dd7/6c7cb77acfa149b0a06da7204440b4f40aa50a0996e6d016c691b989bfe79e7e?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27TinyLlama-1.1B-Chat-v1.0.F16.llamafile%3B+filename%3D%22TinyLlama-1

In [2]:
!pip install -q docx2txt faiss-cpu requests scikit-learn


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [4]:
!ls docs | grep budget

budget_speech.docx


## Step 1: Load the Document

In [1]:
import os
from docx2txt import docx2txt

class SimpleDirectoryLoader(object):
    def __init__(self, directory, allowed_extensions=None):
        self.directory = directory
        self.allowed_extensions = allowed_extensions
        self.handlers = {
            '.docx': self.handle_docx,
        }
        self.data = []
    
    def load_files(self):
        for filename in os.listdir(self.directory):
            ext = os.path.splitext(filename)[-1]
            if (self.allowed_extensions is None or ext in self.allowed_extensions) and ext in self.handlers:
                self.handlers[ext](filename)
            elif self.allowed_extensions is not None and ext not in self.allowed_extensions:
                # print(f"Skipping {filename} with extension {ext}")
                pass
            else:
                print(f"No handler for {ext} files")
        return self.data

    def handle_docx(self, filename):
        text = docx2txt.process(os.path.join(self.directory, filename))
        self.data.append({'filename': filename, 'extension': '.docx', 'data': text})

In [2]:
# Usage
allowed_extensions = ['.docx']
budget_loader = SimpleDirectoryLoader('docs', allowed_extensions=allowed_extensions)
budget_doc = budget_loader.load_files()

# Print first 100 chars of the document
print(budget_doc[0]['data'][:100])

30	



		

		

		Budget 2024-2025

		

		Speech of

		Nirmala Sitharaman

		Minister of Finance

Jul


## Step 2: Chunk the Docs

In [3]:
import re
from typing import List, Dict

class ChunkManager(object):
    def __init__(self, docs: List[Dict[str, str]]):
        self.combined_text = self._combine_texts(docs)
    
    def _combine_texts(self, docs: List[Dict[str, str]]) -> str:
        combined_text = ' '.join(doc['data'] for doc in docs)
        return re.sub(r'\n+', ' ', combined_text)

    def chunk_by_words_with_overlap(self, chunk_size: int, overlap: int) -> List[str]:
        words = self.combined_text.split()
        chunks = []
        step = chunk_size - overlap

        if overlap < 0:
            raise ValueError("Overlap must be a non-negative integer.")
        
        if step <= 0:
            raise ValueError("Overlap must be smaller than chunk size.")

        for i in range(0, len(words), step):
            chunk = ' '.join(words[i:i + chunk_size])
            chunks.append(chunk)
            
        return chunks

In [4]:
# Usage
chunk_size = 250
overlap = 100

chunks = ChunkManager(docs=budget_doc).chunk_by_words_with_overlap(chunk_size, overlap)
print(chunks)

['30 Budget 2024-2025 Speech of Nirmala Sitharaman Minister of Finance July 23, 2024 Hon’ble Speaker, I present the Budget for 2024-25. Introduction The people of India have reposed their faith in the government led by the Hon’ble Prime Minister Shri Narendra Modi and re-elected it for a historic third term under his leadership. We are grateful for their support, faith and trust in our policies. We are determined to ensure that all Indians, regardless of religion, caste, gender and age, make substantial progress in realising their life goals and aspirations. Global Context The global economy, while performing better than expected, is still in the grip of policy uncertainties. Elevated asset prices, political uncertainties and shipping disruptions continue to pose significant downside risks for growth and upside risks to inflation. In this context, India’s economic growth continues to be the shining exception and will remain so in the years ahead. India’s inflation continues to be low, 

## Step 3: Create Embeddings and save in a Vector Store

In [5]:
import os
import faiss
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer

class FAISS(object):
    def __init__(self, max_features=768):
        self.vectorizer = TfidfVectorizer(max_features=max_features)
        self.index = None
        self.texts = []

    # Create Embeddings
    def fit(self, texts):
        self.texts = texts
        embeddings = self.vectorizer.fit_transform(texts).toarray().astype('float32')
        dimension = embeddings.shape[1]
        self.index = faiss.IndexFlatL2(dimension)
        self.index.add(embeddings)
        return embeddings

    # Function for Similarity Search through Context and Query
    # Futher we fetch top k similar documents based on the query
    def search(self, query, k=10):
        query_embedding = self.vectorizer.transform([query]).toarray().astype('float32')
        distances, indices = self.index.search(query_embedding, k)
        similar_texts = [self.texts[idx] for idx in indices[0]]
        return similar_texts

    def save(self, dir_path, index_name='faiss'):
        os.makedirs(dir_path, exist_ok=True)

        with open(os.path.join(dir_path, f'{index_name}.pkl'), 'wb') as f:
            pickle.dump({'vectorizer': self.vectorizer, 'texts': self.texts}, f)
        
        faiss.write_index(self.index, os.path.join(dir_path, f'{index_name}.index'))

    def load(self, dir_path, index_name='faiss'):
        with open(os.path.join(dir_path, f'{index_name}.pkl'), 'rb') as f:
            data = pickle.load(f)
            self.vectorizer = data['vectorizer']
            self.texts = data['texts']
        
        self.index = faiss.read_index(os.path.join(dir_path, f'{index_name}.index'))

In [6]:
# Usage
faiss_index = FAISS()
faiss_index.fit(chunks)

array([[0.        , 0.        , 0.        , ..., 0.03908895, 0.08010402,
        0.07535911],
       [0.        , 0.        , 0.        , ..., 0.03525111, 0.07223923,
        0.13592038],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.06338449],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], dtype=float32)

In [7]:
# Save the embeddings for future use
faiss_index.save('budget_embeddings', index_name='budget_embeddings')

In [8]:
# Load the Budget Index
budget_index = FAISS()
budget_index.load('budget_embeddings', index_name='budget_embeddings')

## Step 4: Similarity Search

In [15]:
# Query the Budget Index for similar documents
# query = "Who represented the Budget 2024?"
query = "Who is the Finance Minister of India?"

no_of_docs_to_fetch = 2    # Get top 2 similar documents
context_chunks = budget_index.search(query, k=no_of_docs_to_fetch)
print(context_chunks)

['30 Budget 2024-2025 Speech of Nirmala Sitharaman Minister of Finance July 23, 2024 Hon’ble Speaker, I present the Budget for 2024-25. Introduction The people of India have reposed their faith in the government led by the Hon’ble Prime Minister Shri Narendra Modi and re-elected it for a historic third term under his leadership. We are grateful for their support, faith and trust in our policies. We are determined to ensure that all Indians, regardless of religion, caste, gender and age, make substantial progress in realising their life goals and aspirations. Global Context The global economy, while performing better than expected, is still in the grip of policy uncertainties. Elevated asset prices, political uncertainties and shipping disruptions continue to pose significant downside risks for growth and upside risks to inflation. In this context, India’s economic growth continues to be the shining exception and will remain so in the years ahead. India’s inflation continues to be low, 

## Step 5: Prompt Engineering

In [16]:
prompt = """You are an expert in the analysis of Government Budgets.
The following comtexts will be from India's Budget 2024.
The user will ask questions about the Budget 2024 and you have to provide the answers based on the context.
Answer in a professional tone and use bullets and paragraphs to make the answer more readable.
If you can't find the answer in the context, reply with "I am sorry, I couldn't find the answer to your question.".
Don't put any external information in the answer.

Context: {context}

Question: {question}
"""

prompt = prompt.format(context=context_chunks, question=query)
print(prompt)

You are an expert in the analysis of Government Budgets.
The following comtexts will be from India's Budget 2024.
The user will ask questions about the Budget 2024 and you have to provide the answers based on the context.
Answer in a professional tone and use bullets and paragraphs to make the answer more readable.
If you can't find the answer in the context, reply with "I am sorry, I couldn't find the answer to your question.".
Don't put any external information in the answer.

Context: ['30 Budget 2024-2025 Speech of Nirmala Sitharaman Minister of Finance July 23, 2024 Hon’ble Speaker, I present the Budget for 2024-25. Introduction The people of India have reposed their faith in the government led by the Hon’ble Prime Minister Shri Narendra Modi and re-elected it for a historic third term under his leadership. We are grateful for their support, faith and trust in our policies. We are determined to ensure that all Indians, regardless of religion, caste, gender and age, make substantia

## Step 6: Run Llamafile on CPU

In [16]:
!chmod +x TinyLlama-1.1B-Chat-v1.0.F16.llamafile

In [ ]:
# Run this command in your computer terminal not in the notebook
# ./TinyLlama-1.1B-Chat-v1.0.F16.llamafile  --nobrowser --server --port 8080 -c 4096

## Chat with the Budget Bot

In [11]:
import requests

class ChatLlamaFile(object):
    def __init__(self, host="localhost", port=8080):
        self.host = host
        self.port = port
        self.base_url = f"http://{host}:{port}"
        self.chat_url = f"{self.base_url}/v1/chat/completions"
        self.headers = {
            'Content-Type': 'application/json',
            'Authorization': 'Bearer no-key'
        }
    
    def chat(self, prompt, query):
        data = {
            "messages": [
                {
                    "role": "system",
                    "content": prompt
                },
                {
                    "role": "user",
                    "content": query
                }
            ]
        }
        response = requests.post(self.chat_url, headers=self.headers, json=data)
        return response.json()

In [17]:
# Let's Chat
response = ChatLlamaFile().chat(prompt, query)
answer = response.get('choices')[0].get('message').get('content')

print("Query:", query)
print("Answer:", answer)

Query: Who is the Finance Minister of India?
Answer: The Finance Minister of India is Nirmala Sitharaman.</s>



[AIMP LABS](https://www.aimplabs.org/) © 2019 - 2024

---